In [5]:
# Daten in die Chromadb Embedden mit Ollama Embedding Modell
import chromadb
import chromadb.utils.embedding_functions as embedding_functions
import ollama

# Verbindung mit ChromaDB herstellen
client = chromadb.HttpClient(host="localhost", port=8095)

# Sammlung erstellen oder abrufen
collection_name = "my_collection"
collection = client.get_or_create_collection(name=collection_name)

# Ollama Embedding Funktion konfigurieren
ollama_ef = embedding_functions.OllamaEmbeddingFunction(
    url="http://localhost:11434/api/embeddings",
    model_name="mxbai-embed-large",
)

# Beispieltexte
pdf_chunks = [
    """
    ### Aufgabe

    Zeichne den Punkt P(4|6) in ein Koordinatensystem ein.

    ### Soll Lösung

    1. Zeichne ein Koordinatensystem mit einer x-Achse und einer y-Achse, damit beide der Punkt im Koordinatensystem liegt. 
       Achte darauf, die Achsen zu beschriften und eine regelmäßige Skalierung zu verwenden.
    2. Markiere den Ursprung (0, 0) des Koordinatensystems deutlich.
    3. Zeichne einen Vektor in x-Richtung, der von (0, 0) bis zum X Punkt reicht (x, 0). 
       Nutze eine klare Farbe wie Rot, um diesen Vektor zu kennzeichnen.
    4. Zeichne einen Vektor in y-Richtung, der von (0, 0) bis zum Y Punkt reicht (0, y). 
       Verwende eine andere Farbe, z. B. Grün.
    5. Verschiebe den y-Richtungs-Vektor so, dass sein Startpunkt das Ende des x-Richtungs-Vektors bei (x, 0) ist.
    6. Der Endpunkt dieses verschobenen Vektors markiert die Koordinaten (x, y). 
       Zeichne einen Punkt an dieser Stelle und beschrifte ihn mit 'P(4|6)'.
    7. Füge ggf. Beschriftungen oder zusätzliche Hinweise hinzu, um die Schritte der Konstruktion verständlicher zu machen.
    """,
    """
    ### Aufgabe: 

    Zeichne einen Kreis mit Mittelpunkt M(0|0) und Radius 5 in ein Koordinatensystem.

    ### Soll Lösung

    1. Zeichne ein Koordinatensystem mit einer x-Achse und einer y-Achse. 
       Achte darauf, die Achsen zu beschriften und eine regelmäßige Skalierung zu verwenden.
    2. Markiere den Ursprung (0, 0) als Mittelpunkt des Kreises.
    3. Zeichne einen Kreis mit einem Radius von 5 Einheiten.
    4. Beschrifte den Mittelpunkt mit 'M(0|0)' und den Radius als 5.
    5. Füge ggf. weitere Beschriftungen hinzu, um die Konstruktion klarer darzustellen.
    """,
    """
    ### Aufgabe: 

    Zeichne die Gerade durch die Punkte A(1|2) und B(4|5) in ein Koordinatensystem.

    ### Soll Lösung

    1. Zeichne ein Koordinatensystem mit einer x-Achse und einer y-Achse. 
       Achte darauf, die Achsen zu beschriften und eine regelmäßige Skalierung zu verwenden.
    2. Markiere die Punkte A(1|2) und B(4|5) im Koordinatensystem.
    3. Verbinde die beiden Punkte mit einer Linie, die durch beide Punkte verläuft. 
       Diese Linie ist die Gerade.
    4. Beschrifte die Punkte A und B mit ihren jeweiligen Koordinaten.
    5. Füge ggf. weitere Hinweise hinzu, um die Schritte klar darzustellen.
    """
]

# Embeddings generieren
embeddings = ollama_ef(pdf_chunks)

# Dokumente vorbereiten
documents = [
    {"text": text, "embedding": embedding}
    for text, embedding in zip(pdf_chunks, embeddings)
]

# Daten zur Sammlung hinzufügen
collection.add(
    ids=[str(i) for i in range(len(documents))],
    documents=[doc["text"] for doc in documents],
    embeddings=[doc["embedding"] for doc in documents]
)

print("Dokumente erfolgreich hinzugefügt.")

# Zähle die Anzahl der Dokumente
document_count = collection.count()

print(f"Anzahl der Dokumente in der Sammlung: {document_count}")

# ChromaDB query function
def query_chromadb(prompt: str) -> str:
    """
    Searches personal documents stored in a ChromaDB collection and generates a response based on the most relevant data.

    Args:
        prompt (str): The search query or question to find related information in your documents.

    Returns:
        str: The most relevant document data or an error message if no data is found.
    """
    try:
        # Generate embedding for the query
        query_embedding = ollama_ef([prompt])[0]
        
        # Perform the query
        results = collection.query(query_embeddings=[query_embedding], n_results=1)
        
        # Extract the most relevant document
        if results['documents']:
            document_data = results['documents'][0][0]
            
        else:
            document_data = "Keine relevanten Daten gefunden."

        response_prompt = f"Using this data: {document_data}. Respond to this prompt: {prompt}"
        response = ollama.generate(model="phi4", prompt=response_prompt)

        return response['response']

    except Exception as e:
        return f"Ein Fehler ist aufgetreten: {e}"



Dokumente erfolgreich hinzugefügt.
Anzahl der Dokumente in der Sammlung: 3


# Ähnliches Problem wie datan im RAG

In [6]:
# Beispielabfrage
user_query = "Zeichne einen Kreis mit Mittelpunkt M(0|0) und Radius 8 in ein Koordinatensystem."
response = query_chromadb(user_query)
print(response)

Um einen Kreis mit dem Mittelpunkt \( M(0,0) \) und einem Radius von 8 Einheiten zu zeichnen, befolgen Sie diese Schritte:

1. **Koordinatensystem Zeichnen:**
   - Zeichne ein Koordinatensystem auf einem Blatt Papier oder einer digitalen Plattform.
   - Füge eine x-Achse und eine y-Achse hinzu, die sich im Ursprung (0, 0) kreuzen.
   - Beschrifte beide Achsen mit "x" und "y".
   - Verwende eine regelmäßige Skalierung auf beiden Achsen. Zum Beispiel könnten Sie jede Einheit durch einen gleichmäßigen Abstand darstellen.

2. **Mittelpunkt Markieren:**
   - Identifiziere den Ursprung des Koordinatensystems als \( M(0,0) \).
   - Markiere diesen Punkt deutlich, da er der Mittelpunkt des Kreises ist.
   - Beschrifte den Punkt mit "M(0|0)".

3. **Kreis Zeichnen:**
   - Verwende einen Zirkel oder ein digitales Werkzeug, um einen Kreis zu zeichnen.
   - Stelle sicher, dass der Mittelpunkt des Zirkels genau auf dem Ursprung \( M(0,0) \) liegt.
   - Setze den Strahl des Zirkels auf 8 Einheiten (v

# Anderes Problem keine Daten im RAG

In [7]:
# Beispielabfrage
user_query = "Zeichne das Dreieck mit den Eckpunkten A(1|1), B(5|1), und C(3|4) in ein Koordinatensystem. Beschrifte die Punkte und verbinde sie."
response = query_chromadb(user_query)
print(response)


Um das Dreieck mit den Eckpunkten A(1,1), B(5,1), und C(3,4) zu zeichnen, folge diesen Schritten:

1. **Koordinatensystem einrichten:**
   - Zeichne zwei sich rechtwinklig schneidende Linien zur Darstellung der x-Achse (waagerecht) und y-Achse (senkrecht).
   - Beschrifte die Achsen als "x" und "y".
   - Wähle eine regelmäßige Skalierung für beide Achsen, z.B. jede Einheit auf dem Papier stellt 1 Einheit im Koordinatensystem dar.

2. **Punkte markieren:**
   - **Punkt A(1,1):** Ziehe von der Ursprungsposition (0,0) eine Strecke von 1 Einheit rechts entlang der x-Achse und dann 1 Einheit nach oben parallel zur y-Achse. Markiere diesen Punkt als A.
   - **Punkt B(5,1):** Ziehe von der Ursprungsposition (0,0) eine Strecke von 5 Einheiten rechts entlang der x-Achse und dann 1 Einheit nach oben parallel zur y-Achse. Markiere diesen Punkt als B.
   - **Punkt C(3,4):** Ziehe von der Ursprungsposition (0,0) eine Strecke von 3 Einheiten rechts entlang der x-Achse und dann 4 Einheiten nach oben 